# 検証用のElasticsearch,Kibanaを構築する

---
検証向けに簡易構成のElasticsearch, Kibanaを構築する。

## 概要

ログの集約や可視化を行うためにElasticsearch, Kibanaを利用することがあります。このnotebookでは検証時の動作確認のために簡易な構成のElasticスタックを構築します。

構築するElasticスタックは以下のような構成としています。

* 1ノード構成
* 認証、TLS接続の無効化
* コンテナによる構築

実運用で利用するには適切な構成ではないので、あくまで検証用としての利用に留めてください。

コンテナを利用して構築を行うので、対象となるホストには事前にdocker engineをインストールしておいてください。

## 準備

このnotebookではansibleを用いて構築を行います。対象となるホストにansibleでアクセスするための準備を行います。

### パラメータの指定

Elasticスタックを構築するホスト名を次のセルで指定してください。

In [ ]:
# (例)
# es_hostname = "es.example.org"
# es_hostname = "172.30.2.150"

es_hostname = 

SSHで接続するためのユーザ名を指定してください。

In [ ]:
# (例)
# ssh_user = "user01"

ssh_user = 

SSHで接続するための秘密鍵を指定してください。

> ペアとなるSSHの公開鍵を事前に対象となるホストに登録しておいてください。

In [ ]:
# (例)
# ssh_private_key_path = "~/.ssh/id_rsa"

ssh_private_key_path = 

構築対象を表すansibleのグループ名を指定してください。

In [ ]:
# (例)
# target_group = "es"

target_group = 

### 設定ファイルの作成

Ansibleのインベントリファイルをカレントディレクトリに`inventory.yml`という名前で作成します。

In [ ]:
from pathlib import Path
import yaml

path_inventory = Path("inventory.yml")
if path_inventory.exists():
    with path_inventory.open() as f:
        inventory = yaml.safe_load(f)
else:
    inventory = {"all": {"children": {}}}
es = {
    target_group: {
        "hosts": {
            es_hostname: {
                "ansible_user": ssh_user,
                "ansible_ssh_private_key_file": str(Path(ssh_private_key_path).expanduser()),
            }
        }
    }
}
inventory["all"]["children"] |= es

with path_inventory.open(mode="w") as f:
    yaml.safe_dump(inventory, f, default_flow_style=False)

!cat inventory.yml

Ansibleの設定ファイル`ansible.cfg`にインベントリファイルのパスを設定します。

In [ ]:
import configparser

config = config = configparser.ConfigParser()
path_cfg = Path("ansible.cfg")
if path_cfg.exists():
    with path_cfg.open() as f:
        config.read_file(f)

if not config.has_section("defaults"):
    config.add_section("defaults")
config["defaults"]["inventory"] = str(path_inventory.resolve())

with path_cfg.open(mode="w") as f:
    config.write(f)

!cat ansible.cfg

`~/.ssh/known_hosts`ファイルを更新します。

In [ ]:
!mkdir -p -m 0700 ~/.ssh
!touch ~/.ssh/known_hosts
!ssh-keygen -R {es_hostname}
!ssh-keyscan -H {es_hostname} >> ~/.ssh/known_hosts

### 接続確認

対象となるホストへの接続確認を行います。

In [ ]:
!ansible {target_group} -m ping

管理者権限でコマンドを実行できることを確認します。

In [ ]:
!ansible {target_group} -b -a 'whoami'

### docker

対象となるホストでdockerが利用できることを確認します。

In [ ]:
!ansible {target_group} -a 'docker info'

## Elasticsearch, Kibana

### パラメータの指定

Elasticスタックを構築するための資材を配置するディレクトリを指定します。

In [ ]:
# (例)
# es_path = "/srv/es"

es_path = "/srv/es"

Elasticスタックのバージョンを指定します。

> このnotebookの構築手順はElasticsearch v9で確認を行なっています。

In [ ]:
# (例)
# es_version = "9.1.2"

es_version = "9.1.2"

設定されたパラメータを保存します。

In [ ]:
%run scripts/group.py
update_group_vars(
    target_group,
    _file='10-es.yml',
    es_path=es_path,
    es_version=es_version,
)

### 資材の配置

Elasticスタックを起動するための資材を配置します。

実際に資材を配置する前にチェックモードで実行します。

In [ ]:
!ansible-playbook -b -l {target_group} -CDv playbooks/deploy-es.yml

実際に資材を配置します。

In [ ]:
!ansible-playbook -b -l {target_group} -v playbooks/deploy-es.yml

### コンテナの実行

コンテナを起動する前に、コンテナイメージの取得を行います。

In [ ]:
!ansible {target_group} -a 'chdir={es_path} docker compose pull -q'

コンテナを起動します。

In [ ]:
!ansible {target_group} -a 'chdir={es_path} docker compose up --wait'

コンテナの状態を確認します。

In [ ]:
!ansible {target_group} -a 'chdir={es_path} docker compose ps'

構築したKibanaにアクセスできることを確認します。

In [ ]:
!curl -fI http://{es_hostname}:5601

Kibanaのアドレスを表示します。

In [ ]:
print(f"http://{es_hostname}:5601")